In [1]:
import tensorflow as tf
import os
import requests
import io
import PIL
import PIL.Image
import math
import pathlib
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Input, Dense, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd



In [2]:
batch_size = 8 # 每⼀批所处理的图⽚数量
img_height = 256 # 图⽚⾼度，单位为像素
img_width = 256 # 图⽚宽度，单位为像素

channels = 3

train_dir = './datasets/tikuimage'

TF_SERVING_BASE_URL = 'http://localhost:8501/'

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.2)
 
training_set = train_datagen.flow_from_directory(
     train_dir,
     target_size = (img_height, img_width),
     batch_size = batch_size,
     class_mode = 'input',
     subset = 'training',
     shuffle=True)
 
validation_set = train_datagen.flow_from_directory(
     train_dir,
     target_size = (img_height, img_width),
     batch_size = batch_size,
     class_mode = 'input',
     subset = 'validation',
     shuffle=False)



Found 54 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [4]:
# Define the autoencoder
input_model = Input(shape=(img_height, img_width, channels))
 
# Encoder layers
encoder = Conv2D(32, (3,3), padding='same', kernel_initializer='normal')(input_model)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization(axis=-1)(encoder)
 
encoder = Conv2D(64, (3,3), padding='same', kernel_initializer='normal')(encoder)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization(axis=-1)(encoder)
 
encoder = Conv2D(64, (3,3), padding='same', kernel_initializer='normal')(input_model)
encoder = LeakyReLU()(encoder)
encoder = BatchNormalization(axis=-1)(encoder)
 
encoder_dim = K.int_shape(encoder)
encoder = Flatten()(encoder)
 
# Latent Space
latent_space = Dense(16, name='latent_space')(encoder)
 
# Decoder Layers
decoder = Dense(np.prod(encoder_dim[1:]))(latent_space)
decoder = Reshape((encoder_dim[1], encoder_dim[2], encoder_dim[3]))(decoder)
 
decoder = Conv2DTranspose(64, (3,3), padding='same', kernel_initializer='normal')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization(axis=-1)(decoder)
 
decoder = Conv2DTranspose(64, (3,3), padding='same', kernel_initializer='normal')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization(axis=-1)(decoder)
 
decoder = Conv2DTranspose(32, (3,3), padding='same', kernel_initializer='normal')(decoder)
decoder = LeakyReLU()(decoder)
decoder = BatchNormalization(axis=-1)(decoder)
 
decoder = Conv2DTranspose(3, (3, 3), padding="same")(decoder)
output = Activation('sigmoid', name='decoder')(decoder)
 
# Create model object
autoencoder = Model(input_model, output, name='autoencoder')
 
# Compile the model
autoencoder.compile(loss="mse", optimizer= Adam(learning_rate=1e-3))
 
# Fit the model
history = autoencoder.fit_generator(
          training_set,
          steps_per_epoch=training_set.n // batch_size,
          epochs=3,
          validation_data=validation_set,
          validation_steps=validation_set.n // batch_size,
          callbacks = [ModelCheckpoint('models/image_autoencoder_2.h5', 
                                       monitor='val_loss', 
                                       verbose=0, 
                                       save_best_only=True, 
                                       save_weights_only=False)])
                                       
 

/Users/sunxiaohua/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
6/6 [==============================] - 17s 3s/step - loss: 0.2790 - val_loss: 0.8813
Epoch 2/3
6/6 [==============================] - 15s 2s/step - loss: 0.2253 - val_loss: 0.6009
Epoch 3/3
6/6 [==============================] - 16s 3s/step - loss: 0.1998 - val_loss: 0.8982


In [5]:
latent_space_model = Model(
                      autoencoder.input, 
                      autoencoder.get_layer('latent_space').output)
 

In [8]:
X = []
indices = []

#Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)。
 
for i in tqdm(range(len(os.listdir('./datasets/tikuimage/new')))):
  try:
    img_name = os.listdir('./datasets/tikuimage')[i]
    img = load_img('./datasets/tikuimage/new/{}'.format(img_name), 
                   target_size = (256, 256))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    pred = latent_space_model.predict(img)
    pred = np.resize(pred, (16))
    X.append(pred)
    indices.append(img_name)
 
  except Exception as e:
    print(img_name)
    print(e)

    # Export the embeddings
embeddings = {'indices': indices, 'features': np.array(X)}
print(embeddings)
#pickle.dump(embeddings, open('./image_embeddings.pickle', 'wb')


<ipython-input-8-7cf31b600ffc>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(os.listdir('./datasets/tikuimage/new')))):


.DS_Store
[Errno 2] No such file or directory: './datasets/tikuimage/new/.DS_Store'
1
[Errno 2] No such file or directory: './datasets/tikuimage/new/1'
new
[Errno 2] No such file or directory: './datasets/tikuimage/new/new'
.ipynb_checkpoints
[Errno 2] No such file or directory: './datasets/tikuimage/new/.ipynb_checkpoints'

{'indices': [], 'features': array([], dtype=float64)}


In [ ]:
print(train_ds.class_names)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
num_classes = 5
model = tf.keras.Sequential([ # 根据需要调整模型结构
layers.experimental.preprocessing.Resizing(img_height, img_width),
tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
   
])

In [ ]:
model.compile(
optimizer='adam',
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [ ]:
model.fit(
train_ds,
validation_data=val_ds,
epochs=1
)
print(model.evaluate(val_ds))

In [ ]:
tf.keras.models.save_model(
model,
'./models/image/1/', # ./models为tensorflow-serving的模型根⽬录
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

In [ ]:
def test_image_model(test_dir, code, batch_size=10): 
  imgs = []
  codes = []
  imgdir = os.path.join(test_dir, str(code)) 
  for i in pathlib.Path(imgdir).glob('./*.jpg'): 
    img = PIL.Image.open(i)
    pixels = np.array(img)
    imgs.append(pixels.tolist()) 
  for i in range(int(math.ceil(len(imgs)/batch_size))):
    req_data = json.dumps({
'     inputs': imgs[i*batch_size:(i+1)*batch_size],
    }) 
    #http://localhost:8501/v1/models/fashion_mnist:predict
    response = requests.post(TF_SERVING_BASE_URL+'v1/models/image/versions/1:predict', # 根据部署地址填写
    data=req_data,
    headers={"content-type": "application/json"})
    if response.status_code != 200:
      raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text) 
    if 'outputs' not in resp_data \
         or type(resp_data['outputs']) is not list:
      raise ValueError('Malformed tf-serving response')
    codes.extend(np.argmax(resp_data['outputs'], axis=1).tolist())
    return codes

In [ ]:
codes = test_image_model(val_ds, 0)
print('类别0的准确率', 1 - round(np.sum(codes)/len(codes),4))
codes = test_image_model(val_ds, 1)
print('类别1的准确率', round(np.sum(codes)/len(codes),4))